In [1]:
import azureml.core
import pandas as pandas
import numpy as np 
import logging

print("AzureML SDK:", azureml.core.VERSION)

AzureML SDK: 1.15.0


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

experiment_name = "automl_bikeshare_forecast"

experimwnt = Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

amlcompute_cluster_name = "cpu-cluster"

provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)

compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

compute_target.wait_for_completion(show_output=True,min_node_count=None, timeout_in_minutes=20)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
